<a href="https://colab.research.google.com/github/xogns1904/webtest/blob/main/%EC%BB%B4%ED%93%A8%ED%84%B0%EB%B9%84%EC%A0%84_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 코랩으로 파일을 업로드하기 (archive.zip으로 저장하였음)
from google.colab import files
uploaded = files.upload()

In [ ]:
# 압축파일 풀기
!unzip archive.zip

In [ ]:
# 필요한 모듈 import하기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os

In [ ]:
# 파일로부터 영상을 읽어서 data에 저장하기


labels = ['PNEUMONIA', 'NORMAL']
# 이미지 크기 결정
img_size = 299
# 파일로부터 영상을 가져오기 (데이터가 저정된 경로)
def get_training_data(data_dir):
    # 이미지 데이터와 해당 클래스 번호를 저장하는 리스트
    data = [] 
    # 리스트의 각 레이블을 반복
    for label in labels: 
        # data_dir과 label을 결합하여 클래스의 경로를 생성
        path = os.path.join(data_dir, label)
        # 현재 클래스의 인덱스를 labels 리스트에서 찾아 class_num 변수에 할당
        class_num = labels.index(label)
        # 클래스 디렉토리 내의 각 이미지 파일을 반복
        for img in os.listdir(path):
            try:
                # cv2를 활용하여 이미지를 그레이스케일로 읽기
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                # Pretrained model로 inception을 쓸 경우 컬러영상으로 변환
                img_arr = cv2.cvtColor(img_arr, cv2.COLOR_GRAY2BGR)
                # img_arr 이미지를 지정된 크기인 (img_size, img_size)로 재조정
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                # 재조정된 이미지와 클래스 번호를 data 리스트에 추가
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    # data 리스트를 np 배열로 변환하여 저장
    return np.array(data)

In [ ]:
ls

In [ ]:
cd chest_xray

In [ ]:
ls

In [ ]:
cd ..

In [ ]:
# 함수를 사용하여 데이터 호출
# train 변수 초기화
train = None

# test 변수 초기화
test = None

# val 변수 초기화
val = None

# train 변수에 train 데이터를 할당
train = get_training_data('./chest_xray/train')
# test 변수에 test 데이터를 할당
test = get_training_data('./chest_xray/test')
# val 변수에 val 데이터를 할당
val = get_training_data('./chest_xray/val')

In [ ]:
# train 변수에 저장된 train 데이터 중 501번째 데이터의 클래스 번호 출력
print(train[500][1])

In [ ]:
# train 데이터의 형태 확인
train.shape

In [ ]:
# train 변수에 저장된 훈련 데이터를 반복하며 각 데이터의 이미지 데이터 feature와 클래스 번호 label을 출력


# 이미지 데이터와 클래스 번호가 저장된 train 배열을 반복하며 각 요소를 feature와 label 변수에 할당
for feature, label in train:
  # 현재 반복되는 데이터의 이미지 데이터를 출력, NumPy 배열로 표현
  print(feature)
  # 현재 반복되는 데이터의 클래스 번호 출력, 클래스 번호 정수로 표현
  print(label)

In [ ]:
#폐렴과 정상 데이터의 갯수 확인하기, train 데이터의 클래스 분포 시각화


# 빈 리스트 I을 생성, 각 데이터의 클래스 저장하기 위해 사용
l = []
# train 배열을 반복하며 각 요소 i를 호출
for i in train:
    # 현재 반복되는 데이터의 클래스 번호가 0인지 확인 (0은 Pneumonia 클래스를 나타낸다.)
    if(i[1] == 0):
        l.append("Pneumonia")
    # 그 외 현재 데이터의 클래스 번호가 0이 아닐 경우, I 리스트에 Normel을 추가
    else:
        l.append("Normal")

# 데이터 프레임 생성
df = pd.DataFrame({"Class": l})

# Seaborn의 countplot 함수를 사용하여 데이터 프레임의 "Class" 열을 계산하고, 각 클래스의 빈도를 시각화
# 이 클래스 분포는 막대 그래프로 나타낸다.
sns.countplot(data=df, x="Class")

# 아래는 오류 발생하여 수정하기 전 코드
# Senborm 패키지를 사용하여 시각화 스타일을 darkgrid로 설정
#sns.set_style('darkgrid')
# Seaborn의 countplot 함수를 사용하여 I 리스트의 요소를 계산하고, 각 클래스의 빈도를 시각화
# 이 클래스 분포는 막대 그래프로 나타낸다.
#sns.countplot(l)  

In [ ]:
#영상으로 보이기, train 데이터의 이미지 시각화


# 새로운 플롯을 생성하고 크기를 5,5로 설정 (플롯 크기 지정)
plt.figure(figsize = (5,5))
# train[0][0]에 저장된 이미지 데이터를 흑백 이미지로 표시
# cmap='gray' 매개변수는 플롯을 그레이스케일로 표시 지정
plt.imshow(train[0][0], cmap='gray')
# train[0][1]에 저장된 클래스 번호를 사용하여 해당 이미지의 제목 설정, 
# labels 리스트에서 해당 클래스 번호에 해당하는 레이블을 가져와 제목 설정
plt.title(labels[train[0][1]])

plt.figure(figsize = (5,5))
plt.imshow(train[-1][0], cmap='gray')
plt.title(labels[train[-1][1]])

In [ ]:
# train 데이터와 test 데이터를 이미지 데이터(feature)와 클래스 번호(label)로 분리하여 저장하는 작업 수행


# train 데이터의 이미지 데이터와 클래스 번호를 저장할 빈 리스트를 생성
x_train = []
y_train = []

# test 데이터의 이미지 데이터와 클래스 번호를 저장할 빈 리스트를 생성
x_test = []
y_test = []

# 위와 동일한 방식
x_val = []
y_val = []

# train 배열을 반복하며 각 요소의 이미지 데이터(feature)와 클래스 번호(label)를 호출
for feature, label in train:
    # 현재 반복되는 train 데이터의 이미지 데이터를 x_train 리스트에 추가
    x_train.append(feature)
    #현재 반복되는 train 데이터의 클래스 번호를 y_train 리스트에 추가
    y_train.append(label)

# test 배열을 반복하며 각 요소의 이미지 데이터(feature)와 클래스 번호(label)를 호출
for feature, label in test:
    # 현재 반복되는 test 데이터의 이미지 데이터를 x_train 리스트에 추가
    x_test.append(feature)
    # 현재 반복되는 test 데이터의 클래스 번호를 y_train 리스트에 추가
    y_test.append(label)

# 위와 동일한 방식
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

In [ ]:
# 데이터 정규화 작업을 수행하여 데이터 처리나 모델 학습에 유리한 상태로 변환


# x_train 리스트를 numpy 배열로 변환한 후, 배열의 모든 요소를 255로 나누어 정규화
# 모든 이미지 데이터의 픽셀 값이 0과 1 사이의 값을 가지도록 조정
# 따라서 train 데이터의 이미지 데이터가 0부터 255 사이의 값을 가진다면 0과 1사이의 값으로 정규화 된다.
# 데이터를 Normalize 0~255  --->   0~1
x_train = np.array(x_train) / 255

# x_test 리스트를 numpy 배열로 변환한 후, 배열의 모든 요소를 255로 나누어 정규화
# test 데이터의 이미지 데이터가 0부터 255 사이의 값을 가진다면 0과 1사이의 값으로 정규화 된다.
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [ ]:
# 이미지 데이터의 형태 변경 작업 수행, 배열의 형태 변경으로 모델 학습에 걸맞는 데이터로 변환
# 딥러닝을 위해 data를 resize


# x_train 배열의 형태를 변경, reshape 함수를 사용하여 배열의 크기를 재조정
# -1은 해당 차원의 크기를 자동으로 계산하도록 지정
# img_size는 이미지의 가로와 세로 크기를 나타냄
# 1은 이미지의 채널 수를 나타냄
x_train = x_train.reshape(-1, img_size, img_size, 3)
# y_train 리스트를 np배열로 변환, 클래스 번호를 np 배열로 변환하여 처리 및 모델 학습에 적용
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 3)
y_val = np.array(y_val)

# 위와 동일 방식
x_test = x_test.reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [ ]:
# data augmentation(데이터 증강)하기 (overfitting을 막고, 데이터셋내의 데이터 불균형도 어느 정도 해소)
# ImageDataGenerator를 설정하고 훈련 데이터에 대해 데이터 증강을 적용


#ImageDataGenerator 클래스를 인스턴스화하여 데이터 증강에 사용할 데이터 생성기를 생성
datagen = ImageDataGenerator(
        # 데이터셋 전체 평균을 0으로 설정
        featurewise_center=False,
        # 각 샘플 평균을 0으로 설정
        samplewise_center=False,
        # 데이터셋 전체의 표준편차로 입력을 정규화
        featurewise_std_normalization=False,
        # 각 샘플의 표준편차로 입력을 정규화
        samplewise_std_normalization=False,
        # ZCA whitening를 적용
        zca_whitening=False,
        # 이미지를 무작위로 회전시키는 범위를 설정, 각도는 0에서 30도까지 무작위 선택
        rotation_range = 30,
        # 이미지를 무작위로 확대 또는 축소하는 범위를 설정, 
        # 0.2는 20%라는 밤위 내에서 무작위로 선택된 확대 또는 축소를 의미
        zoom_range = 0.2,
        # 이미지를 수평으로 무작위로 이동시키는 범위를 설정
        # 0.1은 10% 범위 내에서 무작위로 선택된 이동을 의미 
        width_shift_range=0.1,
        # 이미지를 수직으로 무작위로 이동시키는 범위를 설정
        # 0.1은 10% 범위 내에서 무작위로 선택된 이동을 의미
        height_shift_range=0.1,
        # 이미지를 수평으로 무작위로 뒤집음
        horizontal_flip = True,
        # 이미지를 수직으로 무작위로 뒤집지 않음  
        vertical_flip=False)


# 데이터 증강을 적용하기 전에 데이터 생성기에게 훈련 데이터를 적용하여 내부적으로 몇 가지 통계적 계산을 지시
# 데이터의 평균, 표준편차, 주성분 분석 등의 통계정보, 계산 이를 기반으로 데이터 증강 수행시 적용될 변환 결정
# 훈련 데이터 무작위 변형을 통해 모델의 다양성 및 일반화 성능 향상에 기여
datagen.fit(x_train)

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub

In [ ]:
# GPU 오류 방지를 위해 텐서플로에서 메모리 사용량을 동적으로 관리하는 기능을 활성화한다.


# 텐서플로에서 사용 가능한 GPU 장치의 목록 확인
physical_devices = tf.config.list_physical_devices("GPU")
# GPU 메모리 성장 옵션 활성화
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# keras에서 제공하는 inceptionV3 모델을 사용하여 이미지 분류를 수행


# inceptionV3 모델 생성 (모델은 이미 사전 훈련된 데이터셋에서 가져온 것), include_top=True를 통해 모델의 상단에 전연결층을 포함
model = keras.applications.InceptionV3(include_top=True)
# 모델의 요약(모델의 레이어수, 각 레이어의 입력과 출력 크기, 각 레이어의 활성화 함수 등)을 출력, 모델 아키텍쳐 확인
print(model.summary())

In [ ]:
# =================================================== #
#                Pretrained Keras Model(1)            #
#             classifier부분만 학습                   #
# =================================================== #
# inceptionV3 모델을 기반으로 맨 위의 클래스 분류기 레이어를 제거하고 
# 새로운 클래스 분류기 레이어를 추가하여 새로운 모델을 생성/훈련



# 모델의 입력 레이어를 가져온다.
base_inputs = model.layers[0].input
# 모델의 마지막 레이어 직전의 출력을 가져온다.
base_outputs = model.layers[-2].output
# 가져온 출력을 사용하여 새로운 클래스 분류기 레이어를 생성한다. 
# 이 레이어는 2개의 출력 노드를 가지며 이는 분류 태스크에서 2개의 클래스를 분류하는 것으로 각 출력 노드는 클래스 레이블을 나타낸다.
classifier = layers.Dense(2)(base_outputs)
# inceptionV3 모델의 입력 레이어와 새로운 클래스 분류기 레이어를 연결하여 새로운 모델을 생성한다.
new_model = keras.Model(inputs=base_inputs, outputs=classifier)

# 새로운 모델을 컴파일 한다, 이 단계에서 Adam 옵티마이저(최적화 알고리즘), Sparse Categorical Crossentropy(이진 크로스 엔트로피)손실 함수, 정확도 지표를 설정한다.
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# 새로운 모델의 개요를 출력한다.
print(new_model.summary())
# 새로운 모델을 훈련한다. 이 훈련은 훈련 데이터 셋을 15번 반복하여 수행한다.
new_model.fit(x_train, epochs=15)

In [ ]:
# 훈련 데이터셋에서 (이미지, 레이블)을 가져와 시각화하는 과정을 보여준다.


# 데이터 시각화 라이브러리 matplotlib.pyplot를 임포트하여 plt로 가져옴
import matplotlib.pyplot as plt


# 플롯에 이미지를 나타내기 위해 새로운 plt.figure를 생성하고 figure의 크기는 가로 10, 세로 10으로 설정
plt.figure(figsize=(10, 10))
# 훈련 데이터셋에서 9개의 (이미지, 레이블)을 가져와 반복문을 실행
# 이 때, enumerate() 함수를 사용하여 이미지와 레이블을 가져오면서 해당 이미지의 위치인 인덱스 i를 함께 얻음
for i, (image, label) in enumerate(train_ds.take(9)):
    # 현재 이미지의 레이블을 출력
    print(label) 
    # 3X3 그리드 형태의 서브플롯을 생성 (i+1은 현재 서브플롯의 위치를 나타냄)
    ax = plt.subplot(3, 3, i + 1)
    # 현재 이미지를 imshow() 함수를 사용하여 시각화 (이미지를 플롯에 표시)
    plt.imshow(image)
    # 현재 이미지의 레이블을 정수로 변환하여 플롯의 제목으로 설정
    plt.title(int(label))
    # 플롯의 축(선)을 숨김
    plt.axis("off")

In [ ]:
# 각 데이터셋에 메소드를 순차적으로 적용, 데이터를 전처리하여 모델 학습 및 평가를 효율적으로 수행하도록 한다.


# 데이터셋을 배치로 나눌 때 한 번에 로드되는 샘플의 개수를 지정
# 이 경우 각 데이터셋을 배치로 나눌 때 한 번에 3개의 샘플이 로드되도록 지정
batch_size = 3

# 먼저 cache() 메소드는 데이터셋의 데이터를 캐시에 저장하여 이후 캐시된 데이터는 처음부터 다시 로드하는 것보다 빠르게 엑세스가 가능
# batch(batch_size) 메소드는 데이터 셋의 데이터를 배치로 묶어 지정된 배치 크기에 따라 데이터셋이 분할되어 배치로 로드, 모델 학습 및 평가를 효율적으로 수행
# prefetch(buffer_size=10) 메소드는 데이터셋을 로드하는 동안 데이터셋의 전처리와 모델의 학습/평가가 병렬로 수행되어 속도가 증가
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)